In [1]:
import requests
import pandas as pd
import datetime
from tqdm import tqdm
import time

In [2]:
total_meetings = requests.get("https://api.openf1.org/v1/meetings").json()
session_dict = {}
lap_dict = {}
for meeting in tqdm(total_meetings):
    mk = meeting['meeting_key']
    session_dict[mk] = requests.get(f"https://api.openf1.org/v1/sessions?meeting_key={mk}")
    while (session_dict[mk].status_code != 200):
        time.sleep(0.5)
        session_dict[mk] = requests.get(f"https://api.openf1.org/v1/sessions?meeting_key={mk}")
    session_dict[mk] = session_dict[mk].json()
    for session in session_dict[mk]:
        if session['session_type'] == 'Race':
            session_key = session['session_key']
            lap_dict[mk] = requests.get(f"https://api.openf1.org/v1/laps?session_key={session_key}")
            while (lap_dict[mk].status_code != 200):
                time.sleep(0.5)
                lap_dict[mk] = requests.get(f"https://api.openf1.org/v1/laps?session_key={session_key}")
            lap_dict[mk] = lap_dict[mk].json()
            

100%|██████████| 32/32 [01:26<00:00,  2.70s/it]


In [3]:
for elem in lap_dict.keys():
    if elem not in session_dict.keys(): 
        print(elem in session_dict.keys(), elem)

In [4]:
def get_total_df(lst_dict):
    ret = []
    for k, v in lst_dict.items():
        ret += v
    return pd.DataFrame(ret)

total_laps_df = get_total_df(lap_dict)
# total_laps_df = total_laps_df.loc[:, ["meeting_key", "session_key", "driver_number", "st_speed", "lap_number"]]
total_laps_df.to_csv("laps.csv", index = False)
# total_laps_df.groupby(['meeting_key', 'session_key', 'driver_number']).max().loc[:, "st_speed"]
display(total_laps_df)

,meeting_key,session_key,driver_number,i1_speed,i2_speed,st_speed,date_start,lap_duration,is_pit_out_lap,duration_sector_1,duration_sector_2,duration_sector_3,segments_sector_1,segments_sector_2,segments_sector_3,lap_number
0,1141,7953,1,232.0,231.0,252.0,None,NaN,False,NaN,42.414,23.842,"[2048, 2048, 2049, 2049, 2051, 2051, 2049, 204...","[2051, 2049, 2051, 2049, 2051, 2049, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 0]",1
1,1141,7953,2,224.0,249.0,269.0,None,NaN,False,NaN,45.330,24.706,"[2048, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2048, 2048, 2048, 2048, 2049, 2049]",1
2,1141,7953,4,229.0,236.0,263.0,None,NaN,False,NaN,44.789,24.561,"[2048, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2048, 2048, 2048, 2048, 2049, 2048, 2048]",1
3,1141,7953,10,214.0,234.0,271.0,None,NaN,False,NaN,46.780,25.135,"[2048, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2048, 2048, 2049, 2048, 0]",1
4,1141,7953,11,230.0,237.0,252.0,None,NaN,False,NaN,43.216,24.069,"[2048, 2048, 2049, 2049, 2049, 2049, 2049, 204...","[2049, 2049, 2049, 2049, 2049, 2049, 2049, 204...","[2048, 2048, 2048, 2048, 2048, 2048, 2049]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34244,1237,9531,31,258.0,284.0,326.0,2024-06-09T19:48:13.253000+00:00,77.012,False,22.139,24.500,30.373,"[None, 2048, 2048, 2048, 2049, 2049]","[2048, 2048, 2048, 2048, 2048, 2049]","[2048, 2048, 2048, 0, 0, 0, 0]",70
34245,1237,9531,44,262.0,287.0,334.0,2024-06-09T19:47:50.060000+00:00,74.856,False,21.217,23.745,29.894,"[2051, 2048, 2048, 2048, 2048, 2051]","[2048, 2049, 2049, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2064, 2064]",70
34246,1237,9531,63,258.0,284.0,326.0,2024-06-09T19:47:49.468000+00:00,74.895,False,21.203,23.671,30.021,"[2048, 2048, 2051, 2048, 2049, 2048]","[2048, 2048, 2048, 2051, 2051, 2048]","[2048, 2048, 2048, 2048, 2048, 2064, 2064]",70
34247,1237,9531,77,253.0,281.0,302.0,2024-06-09T19:48:21.450000+00:00,79.042,False,22.472,24.991,31.579,"[None, 2048, 2048, 2048, 2048, 2048]","[2048, 2048, 2048, 2048, 2048, 2048]","[2048, 0, 0, 0, 0, 0, 0]",70


In [5]:
# 문자열을 datetime 객체로 변환하는 함수
def parse_datetime(date_str):
    if '.' in date_str:
        # 소수점이 있는 경우
        return datetime.datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%f%z")
    else:
        # 소수점이 없는 경우
        return datetime.datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S%z")

total_session_df = get_total_df(session_dict)
# total_session_df = total_session_df.loc[:, ["meeting_key", "session_key", "session_type", "date_start"]]
total_session_df['date_start'] = total_session_df['date_start'].map(parse_datetime)
total_session_df

,location,country_key,country_code,country_name,circuit_key,circuit_short_name,session_type,session_name,date_start,date_end,gmt_offset,session_key,meeting_key,year
0,Sakhir,36,BRN,Bahrain,63,Sakhir,Practice,Practice 1,2023-02-23 07:00:00+00:00,2023-02-23T16:30:00+00:00,03:00:00,9222,1140,2023
1,Sakhir,36,BRN,Bahrain,63,Sakhir,Practice,Practice 2,2023-02-24 07:00:00+00:00,2023-02-24T16:30:00+00:00,03:00:00,7763,1140,2023
2,Sakhir,36,BRN,Bahrain,63,Sakhir,Practice,Practice 3,2023-02-25 07:00:00+00:00,2023-02-25T16:30:00+00:00,03:00:00,7764,1140,2023
3,Sakhir,36,BRN,Bahrain,63,Sakhir,Practice,Practice 1,2023-03-03 11:30:00+00:00,2023-03-03T12:30:00+00:00,03:00:00,7765,1141,2023
4,Sakhir,36,BRN,Bahrain,63,Sakhir,Practice,Practice 2,2023-03-03 15:00:00+00:00,2023-03-03T16:00:00+00:00,03:00:00,7766,1141,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,Montréal,46,CAN,Canada,23,Montreal,Practice,Practice 1,2024-06-07 17:30:00+00:00,2024-06-07T18:30:00+00:00,-04:00:00,9524,1237,2024
153,Montréal,46,CAN,Canada,23,Montreal,Practice,Practice 2,2024-06-07 21:00:00+00:00,2024-06-07T22:00:00+00:00,-04:00:00,9525,1237,2024
154,Montréal,46,CAN,Canada,23,Montreal,Practice,Practice 3,2024-06-08 16:30:00+00:00,2024-06-08T17:30:00+00:00,-04:00:00,9526,1237,2024
155,Montréal,46,CAN,Canada,23,Montreal,Qualifying,Qualifying,2024-06-08 20:00:00+00:00,2024-06-08T21:00:00+00:00,-04:00:00,9527,1237,2024


In [6]:
total_meeting_df = pd.DataFrame(total_meetings)
# total_meeting_df = total_meeting_df.loc[:, ["meeting_key", "meeting_name", "country_name", "circuit_short_name", "location", "year"]]
total_meeting_df

,meeting_name,meeting_official_name,location,country_key,country_code,country_name,circuit_key,circuit_short_name,date_start,gmt_offset,meeting_key,year,meeting_code
0,Pre-Season Testing,FORMULA 1 ARAMCO PRE-SEASON TESTING 2023,Sakhir,36,BRN,Bahrain,63,Sakhir,2023-02-23T07:00:00+00:00,03:00:00,1140,2023,NaN
1,Bahrain Grand Prix,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2023,Sakhir,36,BRN,Bahrain,63,Sakhir,2023-03-03T11:30:00+00:00,03:00:00,1141,2023,NaN
2,Saudi Arabian Grand Prix,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2023,Jeddah,153,KSA,Saudi Arabia,149,Jeddah,2023-03-17T13:30:00+00:00,03:00:00,1142,2023,NaN
3,Australian Grand Prix,FORMULA 1 ROLEX AUSTRALIAN GRAND PRIX 2023,Melbourne,5,AUS,Australia,10,Melbourne,2023-03-31T01:30:00+00:00,11:00:00,1143,2023,NaN
4,Azerbaijan Grand Prix,FORMULA 1 AZERBAIJAN GRAND PRIX 2023,Baku,30,AZE,Azerbaijan,144,Baku,2023-04-28T09:30:00+00:00,04:00:00,1207,2023,NaN
5,Miami Grand Prix,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2023,Miami,19,USA,United States,151,Miami,2023-05-05T18:00:00+00:00,-04:00:00,1208,2023,NaN
6,Monaco Grand Prix,FORMULA 1 GRAND PRIX DE MONACO 2023,Monaco,114,MON,Monaco,22,Monte Carlo,2023-05-26T11:30:00+00:00,02:00:00,1210,2023,NaN
7,Spanish Grand Prix,FORMULA 1 AWS GRAN PREMIO DE ESPAÑA 2023,Barcelona,1,ESP,Spain,15,Catalunya,2023-06-02T11:30:00+00:00,02:00:00,1211,2023,NaN
8,Canadian Grand Prix,FORMULA 1 PIRELLI GRAND PRIX DU CANADA 2023,Montréal,46,CAN,Canada,23,Montreal,2023-06-16T17:30:00+00:00,-04:00:00,1212,2023,NaN
9,Austrian Grand Prix,FORMULA 1 ROLEX GROSSER PREIS VON ÖSTERREICH 2023,Spielberg,17,AUT,Austria,19,Spielberg,2023-06-30T15:00:00+00:00,02:00:00,1213,2023,AUT
